In [29]:
import json

# Open the dataset JSON with [r]ead privileges
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipe = ""                # The individual recipe
recipe_list = []           # Where we will store the recipe for training

title = ""                  # The title of the recipe
ingredients_list = []       # The ingredients of the recipe
ingredient_items = []       # The ingredients without units/qtys
instructions = []           # The instructions of the recipe

for meal in data:
    # Get the recipe name and its instructions
    meal_info = data[meal]
    title = meal.replace("-", " ").title() + "\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Sometimes recipes will depend upon already made ingredients, 
        # which need their own set of ingredients.
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = "\n" + i["name"] + ": "
            else: # Otherwise, just add the name
                item = "\n" + i["name"] + " "
        else:
            item = ""

        ingredients_list.append("" + item) # Add the ingredient to the list

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][j] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipes don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient and add it to the list
            # Additionally, add the ingredient's name to the 
            # ingredient_items list
            ingredients_list.append(qty + unit  + ingredient)
            ingredient_items.append(ingredient)
            ingredient = "" # Cleanup
            j += 1

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        instructions.append(str(step) + ". " + instruction)
        step += 1

    # Finally we can construct our recipe
    # We need to add the ingredient names first, as they will represent
    # how the user will input the ingredients for the recipe. After
    # that, we can add the title, ingredients, and instructions.
    recipe = ", ".join(ingredient_items)
    recipe += "|"
    recipe += title  + "\n".join(ingredients_list) + "\n\n" + "\n".join(instructions)

    # Add the recipe to the list
    recipe_list.append(recipe)

    # Free up memory for the next recipe
    title = ""
    ingredients_list = []
    ingredient_items = []
    instructions = []

    meal_info = {}
    recipe = ""
    recipe_instructions = []
    ingredients = []

    item = ""
    qty = ""
    unit = ""
    ingredient = ""
    primary_unit = { "quantity": None, "display": None }

    step = 1

# For testing purposes, we can output a random recipe
import random
print(recipe_list[random.randint(0, len(recipe_list) - 1)])

#print(recipe_list[0])

cream cheese, hot sauce, chicken, shredded monterey jack cheese, mozzarella cheese, refrigerated crescent dinner roll|Cheesy Buffalo Chicken Ring

4 oz cream cheese
¼ cup hot sauce
2 ½ cups chicken
1 cup shredded monterey jack cheese
1 cup mozzarella cheese
8 oz refrigerated crescent dinner roll

1. In a bowl, mix cream cheese and hot sauce until smooth.
2. Mix in chicken and Monterey Jack cheese.
3. Unroll cans of dough, separate into triangles. On a large cookie sheet, arrange the triangles in ring, dough should overlap, leaving around 5 inches (12 ¾ cm) in the center.
4. Layer half of the Mozzarella on each triangle closest to the ring.
5. Spoon mixture on top of the Mozzarella. Top with the rest of the Mozzarella.
6. Bring the top of the triangle over the filling and tuck the edges under the bottom layer of dough. Repeat around the ring until the entire filling is enclosed.
7. Bake at 375˚F (190˚C) for 35 minutes or until the dough is golden brown.
8. Cool 5 to 10 minutes before cu

In [30]:
print(recipe_list[0])

unsalted butter, whole milk, granulated sugar, active dry yeast, flour, baking powder, salt, butter, light brown sugar, ground cinnamon, cream cheese, butter, whole milk, vanilla extract, powdered sugar|Homemade Cinnamon Rolls

Dough: 
½ cup unsalted butter
2 cups whole milk
½ cup granulated sugar
2 ¼ teaspoons active dry yeast
5 cups flour
1 teaspoon baking powder
2 teaspoons salt

Filling: 
¾ cup butter
¾ cup light brown sugar
2 tablespoons ground cinnamon

Frosting: 
4 oz cream cheese
2 tablespoons butter
2 tablespoons whole milk
1 teaspoon vanilla extract
1 cup powdered sugar

1. Generously butter two disposable foil pie/cake pans.
2. In a large bowl, whisk together warm milk, melted butter, and granulated sugar. The mixture should be just warm, registering between 100-110˚F (37-43˚C). If it is hotter, allow to cool slightly.
3. Sprinkle the yeast evenly over the warm mixture and let set for 1 minute.
4. Add 4 cups (500g) of all-purpose flour to the milk mixture and mix with a wood

In [31]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe
# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe_GPT3.csv", 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["prompt","completion"])
    for recipe in recipe_list:
        newrecipe=recipe.split("|")
        csvwriter.writerow([newrecipe[0],newrecipe[1]])